In [34]:
import Pkg
#Pkg.add("JuMP")
#Pkg.add("GLPK")
#Pkg.add("Gruobi")
#Pkg.add("DataFrames")
#Pkg.add("CSV")
#Pkg.add("PyCall")

In [35]:
using JuMP, GLPK
using DataFrames
using CSV
using PrettyTables

In [36]:
# Load the data
scenarios_df = CSV.read("../data/scenarios.csv", DataFrame)

n_scenarios = size(scenarios_df, 2)/3
n_scenarios = convert(Int, n_scenarios)

# create a dictonary with 200 dataframes for each scenario
scenarios = Dict()
for i in 1:n_scenarios
    df_helper = DataFrame(scenarios_df[:,3*i-2:3*i])
    df_helper[!,3] = df_helper[!,3] .* 1.0
    rename!(df_helper, [:"price", :"wind power", :"grid_excess"])
    scenarios[i] = df_helper
end

In [37]:
W = 250
hours = 24

24

In [38]:
# Create a new model with GLPK solver
model = Model(GLPK.Optimizer)

# Define the decision variables for hour
@variable(model, p_DA[1:hours])
@variable(model, delta[1:W,1:hours])

# Define the objective function
@objective(model, Max, sum(1/W .*
(scenarios[i][hour,"price"] * p_DA[hour]
+ scenarios[i][hour,"price"] * delta[i, hour]*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"]))) for i in 1:W, hour in 1:hours))

# Define the constraints
@constraint(model, [hour in 1:hours], p_DA[hour] <= 200)
@constraint(model, [hour in 1:hours], p_DA[hour] >= 0)
@constraint(model, [i in 1:W, hour in 1:hours], delta[i,hour] == scenarios[i][hour,"wind power"] - p_DA[hour])


# Solve the optimization problem
optimize!(model)


#println(model)

# Print the termination status
status = termination_status(model)
if status == MOI.OPTIMAL
    println("Optimal solution found")
    
    # RETURN OBJECTIVE value
    println("Objective value: ", objective_value(model))
else
    println("No optimal solution found")
end

Optimal solution found
Objective value: 199651.90331717237


In [39]:
println("Objective value: ", objective_value(model))
println("p_DA: ", value.(p_DA))
println("delta: ", value.(delta))

Objective value: 199651.90331717237
p_DA: [200.0, 0.0, 0.0, 0.0, 200.0, 200.0, 200.0, 0.0, 0.0, 200.0, 0.0, 0.0, 200.0, 200.0, 200.0, 200.0, 200.0, 200.0, 0.0, 200.0, 0.0, 200.0, 0.0, 200.0]
delta: [-85.22370967741936 117.26580643548388 117.01693548387098 113.00290319354838 -91.85338714516128 -90.5651613064516 -95.65080646774196 103.62580643548388 103.5698387096774 -95.1154838870968 110.126129 110.24129032258064 -94.8140322903226 -87.60806451612903 -83.91870970967742 -87.00322582258065 -85.7767741935484 -83.21596777419356 118.55774187096776 -83.718871016129 107.72741933870968 -93.5274193548387 105.13532258064518 -94.54112904838708; -85.22370967741936 117.26580643548388 117.01693548387098 113.00290319354838 -91.85338714516128 -90.5651613064516 -95.65080646774196 103.62580643548388 103.5698387096774 -95.1154838870968 110.126129 110.24129032258064 -94.8140322903226 -87.60806451612903 -83.91870970967742 -87.00322582258065 -85.7767741935484 -83.21596777419356 118.55774187096776 -83.71887101

In [40]:
objective_value(model)

199651.90331717237

In [41]:
# Initialize the DataFrame directly without dynamic column names
result_df = DataFrame(hour = Int[], p_DA = Float64[])

# make a table with the results
for hour in 1:hours
    push!(result_df, [hour, value(p_DA[hour])])
end

profit_df = DataFrame(scenario = Int[], profit = Float64[])

for i in 1:W
    profit = 0
    for hour in 1:hours
        profit += (scenarios[i][hour,"price"] * value(p_DA[hour])
        + scenarios[i][hour,"price"] * value(delta[i, hour])*((0.9.*scenarios[i][hour,"grid_excess"]) + 1.2.*(1-scenarios[i][hour,"grid_excess"])))
    end
    push!(profit_df, [i, profit])
end

In [42]:
pretty_table(result_df, backend = Val(:latex))

\begin{tabular}{rr}
  \hline
  \textbf{hour} & \textbf{p\_DA} \\
  \texttt{Int64} & \texttt{Float64} \\\hline
  1 & 200.0 \\
  2 & 0.0 \\
  3 & 0.0 \\
  4 & 0.0 \\
  5 & 200.0 \\
  6 & 200.0 \\
  7 & 200.0 \\
  8 & 0.0 \\
  9 & 0.0 \\
  10 & 200.0 \\
  11 & 0.0 \\
  12 & 0.0 \\
  13 & 200.0 \\
  14 & 200.0 \\
  15 & 200.0 \\
  16 & 200.0 \\
  17 & 200.0 \\
  18 & 200.0 \\
  19 & 0.0 \\
  20 & 200.0 \\
  21 & 0.0 \\
  22 & 200.0 \\
  23 & 0.0 \\
  24 & 200.0 \\\hline
\end{tabular}


In [43]:
CSV.write("1_1_results/profits_1_1.csv", profit_df)
CSV.write("1_1_results/market_results_1_1.csv", result_df)

"1_1_results/market_results_1_1.csv"